In [111]:
import pandas as pd
pd.set_option('display.max_rows', 100)

In [112]:
circuits_df = pd.read_csv("../datasets/circuits.csv")
race_df = pd.read_csv("../datasets/races.csv")
country_codes_df = pd.read_csv("../datasets/country_codes.csv")
results_df = pd.read_csv("../datasets/results.csv")
drivers_df = pd.read_csv("../datasets/drivers.csv")
driver_st_df = pd.read_csv("../datasets/driver_standings.csv")
constructors_df = pd.read_csv("../datasets/constructors.csv")
constructors_results_df = pd.read_csv("../datasets/constructors_results.csv")

In [ ]:
circuit_race_counts = race_df.value_counts(subset=['circuitId']).to_frame('count').reset_index()
df_merge = pd.merge(circuits_df, circuit_race_counts, on='circuitId', how='left')
country_counts = df_merge.groupby(['country']).sum().reset_index()
df = pd.merge(country_counts, country_codes_df, on='country', how='left')

# df

In [ ]:
country_circuits = circuits_df[['circuitId', 'country', 'name', 'location']].loc[circuits_df['country']=='Canada']
country_circuits.rename({"name": 'circuit_name'}, axis=1, inplace=True)
df = pd.merge(country_circuits, race_df, on='circuitId', how='left')
df.sort_values('year', inplace=True)
df.reset_index(drop=True, inplace=True)
# df = df[["name", "location", "year", "circuit_name", "url"]]
df

In [130]:
import numpy as np

ddfsf = pd.merge(driver_st_df, race_df[['year', 'raceId']], on='raceId', how='left')

# driver_wins = pd.pivot_table(driver_st_df, index=['driverId'],values=['wins'],aggfunc=np.max).groupby('driverId').agg({'wins':'sum'}).reset_index().sort_values('wins',ascending=False)
# pd.merge(driver_wins, drivers_df, on='driverId', how='inner').head(10)
# driver_wins
dfdfdf = ddfsf.groupby(['year','driverId']).agg({'wins':np.max}).groupby('driverId').agg({'wins':'sum'}).sort_values('wins',ascending=False)
pd.merge(dfdfdf, drivers_df, on='driverId', how='inner').head(10)
# ddfsf

,driverId,wins,driverRef,number,code,forename,surname,dob,nationality,url
0,1,103,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,30,91,michael_schumacher,\N,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher
2,20,53,vettel,5,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel
3,117,51,prost,\N,\N,Alain,Prost,1955-02-24,French,http://en.wikipedia.org/wiki/Alain_Prost
4,102,41,senna,\N,\N,Ayrton,Senna,1960-03-21,Brazilian,http://en.wikipedia.org/wiki/Ayrton_Senna
5,4,32,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
6,95,31,mansell,\N,\N,Nigel,Mansell,1953-08-08,British,http://en.wikipedia.org/wiki/Nigel_Mansell
7,328,27,stewart,\N,\N,Jackie,Stewart,1939-06-11,British,http://en.wikipedia.org/wiki/Jackie_Stewart
8,182,25,lauda,\N,\N,Niki,Lauda,1949-02-22,Austrian,http://en.wikipedia.org/wiki/Niki_Lauda
9,373,25,clark,\N,\N,Jim,Clark,1936-03-04,British,http://en.wikipedia.org/wiki/Jim_Clark


In [23]:
from datetime import datetime
import time

df2 = pd.merge(results_df, df, on='raceId', how='inner')
print(df2.fastestLapTime[0])

datetime_object = datetime.strptime(df2.fastestLapTime[0], '%M:%S.%f')
# time.mktime(datetime_object)
# datetime_object.minute *60 + datetime_object.second + datetime_object.microsecond /1000000
datetime_object

1:17.539


datetime.datetime(1900, 1, 1, 0, 1, 17, 539000)

In [24]:
df2['fastestLapTime'] = df2['fastestLapTime'].apply(lambda x: datetime.strptime(x, '%M:%S.%f').time() if x != '\\N' else datetime.strptime("10:0.0", '%M:%S.%f').time())
min = df2['fastestLapTime'].min()
idxmin = df2[df2['fastestLapTime'] == min]
min.minute *60 + min.second + min.microsecond /1000000
driverId = idxmin['driverId'].values[0]
driverIdx = drivers_df[drivers_df['driverId'] == driverId]
forename = driverIdx['forename']
surname = driverIdx['surname']
year = idxmin['year'].values[0]
f"{forename.values[0]} {surname.values[0]} ({year})"

'Valtteri Bottas (2019)'

In [205]:
df2 = pd.merge(results_df, df, on='raceId', how='inner')
df3 = df2.loc[df2['positionText'] == '1']
df4 = df3.value_counts(subset=['driverId']).to_frame('count').reset_index()
df5 = pd.merge(df4, drivers_df, on='driverId', how='inner')
df6 = df5.loc[df5['count'] == df5['count'].max()]

fastest_drivers = ""
for i in range(len(df6)):
    forename = df6['forename'].values[i]
    surname = df6['surname'].values[i]
    count = df6['count'].values[i]
    fastest_drivers += f"{forename} {surname} ({int(count)})"
    if i != len(df6)-1:
        fastest_drivers += ", "

fastest_drivers

'Lewis Hamilton (7), Michael Schumacher (7)'

In [204]:
df2 = pd.merge(results_df, df, on='raceId', how='inner')
df3 = df2.loc[df2['positionText'] == '1']
df4 = df3.value_counts(subset=['constructorId']).to_frame('count').reset_index()
df5 = pd.merge(df4, drivers_df, on='constructorId', how='inner')
df6 = df5.loc[df5['count'] == df5['count'].max()]

fastest_drivers = ""
for i in range(len(df6)):
    forename = df6['forename'].values[i]
    surname = df6['surname'].values[i]
    count = df6['count'].values[i]
    fastest_drivers += f"{forename} {surname} ({int(count)})"
    if i != len(df6)-1:
        fastest_drivers += ", "

fastest_drivers

'Lewis Hamilton (7), Michael Schumacher (7)'

In [206]:
results_df

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25394,25400,1073,815,9,11,4,15,15,15,0.0,55,\N,\N,51,2,1:26.419,219.993,5
25395,25401,1073,849,3,6,16,\N,R,16,0.0,50,\N,\N,30,15,1:29.293,212.912,3
25396,25402,1073,841,51,99,14,\N,R,17,0.0,33,\N,\N,33,16,1:29.442,212.557,6
25397,25403,1073,847,3,63,17,\N,R,18,0.0,26,\N,\N,23,19,1:30.647,209.732,6
